In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import cv2
import seaborn as sns
from tqdm import tqdm
import pandas as pd

def load_model_and_create_feature_extractor(model_path):
    """
    Load the trained model and create a feature extractor
    
    Args:
        model_path: Path to the trained model
    
    Returns:
        Feature extractor model
    """
    # Load the model
    model = load_model(model_path)
    
    # Create a feature extractor model (using the output before the final classification layer)
    # For the provided model architecture, this is the output after GlobalAveragePooling2D
    feature_layer = 'global_average_pooling2d'
    
    # Create a new model that outputs features
    feature_extractor = Model(
        inputs=model.input,
        outputs=model.get_layer(feature_layer).output
    )
    
    return feature_extractor

In [2]:
def extract_features(feature_extractor, image_dir, batch_size=32):
    """
    Extract features from images in a directory
    
    Args:
        feature_extractor: Model for feature extraction
        image_dir: Directory containing images
        batch_size: Batch size for processing
    
    Returns:
        Features, labels, and filenames
    """
    # Data generator for feature extraction
    datagen = ImageDataGenerator(rescale=1./255)
    
    # Create generator
    generator = datagen.flow_from_directory(
        image_dir,
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False  # Don't shuffle to keep track of filenames
    )
    
    # Extract features
    features = []
    labels = []
    filenames = []
    
    # Get class names
    class_names = list(generator.class_indices.keys())
    
    # Calculate steps
    steps = np.ceil(generator.samples / batch_size).astype(int)
    
    print(f"Extracting features from {generator.samples} images...")
    
    # Process batches
    for i in tqdm(range(steps)):
        batch_images, batch_labels = next(generator)
        batch_features = feature_extractor.predict(batch_images)
        
        features.append(batch_features)
        labels.append(np.argmax(batch_labels, axis=1))
        filenames.extend([os.path.basename(f) for f in generator.filenames[i*batch_size:(i+1)*batch_size]])
    
    # Concatenate batches
    features = np.vstack(features)
    labels = np.concatenate(labels)
    
    return features, labels, filenames, class_names


## tsne 

In [3]:
def visualize_tsne(features, labels, class_names, output_path=None):
    """
    Visualize features using t-SNE
    
    Args:
        features: Extracted features
        labels: Class labels
        class_names: Names of classes
        output_path: Path to save the visualization
    """
    # Apply t-SNE
    print("Applying t-SNE dimensionality reduction...")
    tsne = TSNE(n_components=2, perplexity=30, n_iter=1000, random_state=42)
    features_tsne = tsne.fit_transform(features)
    
    # Visualize
    plt.figure(figsize=(12, 10))
    
    # Plot each class
    for i, class_name in enumerate(class_names):
        idx = labels == i
        plt.scatter(features_tsne[idx, 0], features_tsne[idx, 1], label=class_name, alpha=0.7)
    
    plt.title('t-SNE Visualization of Image Features')
    plt.xlabel('t-SNE Dimension 1')
    plt.ylabel('t-SNE Dimension 2')
    plt.legend(title='Classes')
    plt.grid(alpha=0.3)
    
    # Save if output path is provided
    if output_path:
        plt.savefig(output_path)
    
    plt.show()


## PCA 

In [4]:
def visualize_pca(features, labels, class_names, output_path=None):
    """
    Visualize features using PCA
    
    Args:
        features: Extracted features
        labels: Class labels
        class_names: Names of classes
        output_path: Path to save the visualization
    """
    # Apply PCA
    print("Applying PCA dimensionality reduction...")
    pca = PCA(n_components=2)
    features_pca = pca.fit_transform(features)
    
    # Visualize
    plt.figure(figsize=(12, 10))
    
    # Plot each class
    for i, class_name in enumerate(class_names):
        idx = labels == i
        plt.scatter(features_pca[idx, 0], features_pca[idx, 1], label=class_name, alpha=0.7)
    
    plt.title('PCA Visualization of Image Features')
    plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.2%} explained variance)')
    plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.2%} explained variance)')
    plt.legend(title='Classes')
    plt.grid(alpha=0.3)
    
    # Save if output path is provided
    if output_path:
        plt.savefig(output_path)
    
    plt.show()

In [5]:
def visualize_activation_maps(model, image_path, class_names, output_path=None):
    """
    Visualize activation maps (Grad-CAM) for a given image
    
    Args:
        model: Trained model
        image_path: Path to image
        class_names: Class names
        output_path: Path to save visualization
    """
    # Load and preprocess the image
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_resized = cv2.resize(img, (224, 224))
    
    # Apply preprocessing
    img_yuv = cv2.cvtColor(img_resized, cv2.COLOR_RGB2YUV)
    img_yuv[:,:,0] = cv2.equalizeHist(img_yuv[:,:,0])
    img_eq = cv2.cvtColor(img_yuv, cv2.COLOR_YUV2RGB)
    img_blur = cv2.GaussianBlur(img_eq, (3, 3), 0)
    
    # Normalize
    img_normalized = img_blur.astype(np.float32) / 255.0
    img_batch = np.expand_dims(img_normalized, axis=0)
    
    # Get predictions
    predictions = model.predict(img_batch)[0]
    predicted_class = np.argmax(predictions)
    confidence = predictions[predicted_class]
    
    # Create Grad-CAM heatmap
    import tensorflow as tf
    
    # Find the last convolutional layer
    last_conv_layer = None
    for layer in reversed(model.layers):
        if isinstance(layer, tf.keras.layers.Conv2D):
            last_conv_layer = layer.name
            break
    
    if last_conv_layer is None:
        print("Could not find a convolutional layer in the model")
        return
    
    # Create a model that outputs both the predictions and the activations
    grad_model = Model(
        inputs=model.inputs,
        outputs=[model.get_layer(last_conv_layer).output, model.output]
    )
    
    # Compute gradient of the predicted class with respect to the activations
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_batch)
        class_prediction = predictions[:, predicted_class]
    
    # Gradients of the predicted class with respect to the output feature map
    grads = tape.gradient(class_prediction, conv_outputs)
    
    # Pool the gradients across the channels
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    
    # Weight the channels by the gradient values
    conv_outputs = conv_outputs[0]
    heatmap = tf.reduce_sum(tf.multiply(pooled_grads, conv_outputs), axis=-1)
    
    # Normalize the heatmap
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    heatmap = heatmap.numpy()
    
    # Resize heatmap to original image size
    heatmap = cv2.resize(heatmap, (img_resized.shape[1], img_resized.shape[0]))
    
    # Convert heatmap to RGB
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    
    # Superimpose heatmap on original image
    superimposed_img = heatmap * 0.4 + img_resized
    superimposed_img = np.clip(superimposed_img, 0, 255).astype('uint8')
    
    # Display
    plt.figure(figsize=(15, 5))
    
    # Original image
    plt.subplot(1, 3, 1)
    plt.imshow(img_resized)
    plt.title('Original Image')
    plt.axis('off')
    
    # Heatmap
    plt.subplot(1, 3, 2)
    plt.imshow(heatmap)
    plt.title('Activation Heatmap')
    plt.axis('off')
    
    # Superimposed
    plt.subplot(1, 3, 3)
    plt.imshow(superimposed_img)
    plt.title(f'Prediction: {class_names[predicted_class]}\nConfidence: {confidence:.2%}')
    plt.axis('off')
    
    plt.tight_layout()
    
    if output_path:
        plt.savefig(output_path)
    
    plt.show()

## Misclassified 

In [6]:
def find_misclassified(model, dataset_dir, output_dir=None, batch_size=32):
    """
    Find and visualize misclassified images
    
    Args:
        model: Trained model
        dataset_dir: Directory containing test images
        output_dir: Directory to save visualizations
        batch_size: Batch size for processing
    
    Returns:
        DataFrame with misclassification details
    """
    # Create output directory if needed
    if output_dir and not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Data generator
    datagen = ImageDataGenerator(rescale=1./255)
    
    # Get test generator
    test_dir = os.path.join(dataset_dir, 'test')
    test_generator = datagen.flow_from_directory(
        test_dir,
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False
    )
    
    # Get class names
    class_names = list(test_generator.class_indices.keys())
    
    # Make predictions
    steps = np.ceil(test_generator.samples / batch_size).astype(int)
    predictions = model.predict(test_generator, steps=steps)
    predicted_classes = np.argmax(predictions, axis=1)
    
    # Get true labels
    true_classes = test_generator.classes[:len(predicted_classes)]
    
    # Find misclassified images
    misclassified_indices = np.where(predicted_classes != true_classes)[0]
    
    if len(misclassified_indices) == 0:
        print("No misclassified images found!")
        return None
    
    # Prepare results
    misclassified_results = []
    
    print(f"Found {len(misclassified_indices)} misclassified images")
    
    # Process each misclassified image
    for idx in misclassified_indices:
        true_class = true_classes[idx]
        pred_class = predicted_classes[idx]
        confidence = predictions[idx][pred_class]
        image_path = test_generator.filepaths[idx]
        image_name = os.path.basename(image_path)
        
        misclassified_results.append({
            'image': image_name,
            'true_class': class_names[true_class],
            'predicted_class': class_names[pred_class],
            'confidence': confidence,
            'file_path': image_path
        })
        
        # Visualize if output directory is provided
        if output_dir:
            # Load and preprocess image
            img = cv2.imread(image_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            
            # Create visualization
            plt.figure(figsize=(8, 6))
            plt.imshow(img)
            plt.title(f"True: {class_names[true_class]}\nPredicted: {class_names[pred_class]} ({confidence:.2%})")
            plt.axis('off')
            plt.tight_layout()
            
            # Save
            save_path = os.path.join(output_dir, f"misclassified_{image_name}")
            plt.savefig(save_path)
            plt.close()
    
    # Convert to DataFrame
    results_df = pd.DataFrame(misclassified_results)
    
    # Save results to CSV
    if output_dir:
        results_df.to_csv(os.path.join(output_dir, 'misclassified_images.csv'), index=False)
    
    return results_df


## main function 

In [7]:
def run_cnn_visualization(model_path, dataset_path, output_dir='visualization_results', 
                         image_path=None, mode='all'):
    """
    Run CNN visualization with the specified parameters.
    
    Args:
        model_path: Path to the trained model
        dataset_path: Path to the dataset directory
        output_dir: Directory to save results (default: 'visualization_results')
        image_path: Path to a specific image for activation map visualization (optional)
        mode: Visualization mode (options: 'tsne', 'pca', 'activation', 'misclassified', 'all')
    """
    # Create output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Load model
    print(f"Loading model from {model_path}...")
    model = load_model(model_path)
    
    # Get class names
    # Try to load class names from a file next to the model
    class_names_path = os.path.join(os.path.dirname(model_path), 'class_names.txt')
    
    if os.path.exists(class_names_path):
        with open(class_names_path, 'r') as f:
            class_names = [line.strip() for line in f.readlines()]
        print(f"Loaded {len(class_names)} classes from file: {class_names}")
    else:
        # If class names file doesn't exist, try to get them from the test directory
        test_dir = os.path.join(dataset_path, 'test')
        if os.path.exists(test_dir):
            datagen = ImageDataGenerator(rescale=1./255)
            generator = datagen.flow_from_directory(
                test_dir,
                target_size=(224, 224),
                batch_size=1,
                class_mode='categorical',
                shuffle=False
            )
            class_names = list(generator.class_indices.keys())
            print(f"Loaded {len(class_names)} classes from test directory: {class_names}")
        else:
            print("Warning: Could not find class names file or test directory.")
            # Default to numeric class names
            num_classes = model.layers[-1].output_shape[-1]
            class_names = [f"Class {i}" for i in range(num_classes)]
            print(f"Using default class names: {class_names}")
    
    # Run visualizations based on mode
    if mode in ['tsne', 'pca', 'all']:
        # Extract features from test images
        feature_extractor = load_model_and_create_feature_extractor(model_path)
        test_dir = os.path.join(dataset_path, 'test')
        
        features, labels, filenames, _ = extract_features(
            feature_extractor, 
            test_dir
        )
        
        # t-SNE visualization
        if mode in ['tsne', 'all']:
            print("Generating t-SNE visualization...")
            tsne_output_path = os.path.join(output_dir, 'tsne_visualization.png')
            visualize_tsne(features, labels, class_names, tsne_output_path)
        
        # PCA visualization
        if mode in ['pca', 'all']:
            print("Generating PCA visualization...")
            pca_output_path = os.path.join(output_dir, 'pca_visualization.png')
            visualize_pca(features, labels, class_names, pca_output_path)
    
    # Activation map visualization
    if mode in ['activation', 'all'] and image_path:
        print(f"Generating activation map for {image_path}...")
        activation_output_path = os.path.join(output_dir, 'activation_map.png')
        visualize_activation_maps(model, image_path, class_names, activation_output_path)
    
    # Find misclassified images
    if mode in ['misclassified', 'all']:
        print("Finding misclassified images...")
        misclassified_output_path = os.path.join(output_dir, 'misclassified')
        if not os.path.exists(misclassified_output_path):
            os.makedirs(misclassified_output_path)
        
        misclassified_results = find_misclassified(
            model, 
            dataset_path, 
            misclassified_output_path
        )
    
    print("Visualization complete!")
    return "All visualizations completed successfully"


In [15]:
# Set your paths
MODEL_PATH = '/Users/anamikasaroha/Energy7_Week1/week_5/model/final_model.h5'
DATASET_PATH = '/Users/anamikasaroha/Energy7_Week1/week_5/dataset_split'
OUTPUT_DIR = 'visualization_results'
IMAGE_PATH = '/Users/anamikasaroha/Energy7_Week1/week_5/dataset_split/test/159_160/sample_129_159_160.png'  # Optional

In [16]:
run_cnn_visualization(MODEL_PATH, DATASET_PATH, OUTPUT_DIR, IMAGE_PATH, mode='misclassified')

Loading model from /Users/anamikasaroha/Energy7_Week1/week_5/model/final_model.h5...


Loaded 16 classes from file: ['129_130', '159_160', 'PT101_102', 'PT119_120', 'PT_101_102', 'PT_101___102', 'PT_103_104', 'PT_109_110', 'PT_111___112', 'PT_119_120', 'PT_119___120', 'PT_121_122', 'PT_129_130', 'PT_135___136', 'PT_157___158', 'PT_189___190']
Finding misclassified images...
Found 240 images belonging to 16 classes.


/opt/anaconda3/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


8/8 ━━━━━━━━━━━━━━━━━━━━ 8s 841ms/step
Found 117 misclassified images
Visualization complete!


'All visualizations completed successfully'

In [19]:
run_cnn_visualization(MODEL_PATH, DATASET_PATH, OUTPUT_DIR, IMAGE_PATH, mode='activation')

Loading model from /Users/anamikasaroha/Energy7_Week1/week_5/model/final_model.h5...


Loaded 16 classes from file: ['129_130', '159_160', 'PT101_102', 'PT119_120', 'PT_101_102', 'PT_101___102', 'PT_103_104', 'PT_109_110', 'PT_111___112', 'PT_119_120', 'PT_119___120', 'PT_121_122', 'PT_129_130', 'PT_135___136', 'PT_157___158', 'PT_189___190']
Generating activation map for /Users/anamikasaroha/Energy7_Week1/week_5/dataset_split/test/159_160/sample_129_159_160.png...
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Could not find a convolutional layer in the model
Visualization complete!


'All visualizations completed successfully'